In [1]:
pip install opencv-python



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /usr/local/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [87]:
import cv2
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd

In [88]:
file_path = './labwork5dataset/lab6.xlsx'
df = pd.read_excel(file_path)

variant_number = ord("D") % 5 + 1
variant_data = df[df['N'] == variant_number].iloc[0]
print(variant_data)

image_filename = variant_data['file name']
glasses_color = variant_data['glasses color']
line_width = variant_data['line width']
output_size = tuple(map(int, variant_data['image size'].split('x')))

image_path = f'./labwork5dataset/images/{image_filename}'
output_path_with_brackets = './labwork5dataset/longbottom_with_glasses_and_brackets.jpg'

N                             4
file name        longbottom.jpg
image size              600x600
glasses color           magenta
line width                    5
Name: 3, dtype: object


In [95]:
def draw_glasses_with_brackets(image_path, output_path, glasses_color="magenta", line_width=5):
    # Load the image and convert to grayscale for face and eye detection
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Load Haar Cascade for face and eye detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    
    if len(faces) == 0:
        print("No face detected.")
        return
    
    # We assume only one face in the image for simplicity
    for (x, y, w, h) in faces:
        # Center face in the output image if needed
        face_center_x, face_center_y = x + w // 2, y + h // 2
        # Detect eyes within the face region
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # We assume two eyes detected
        if len(eyes) < 2:
            print("Could not detect both eyes.")
            return
        
        # Sort eyes by x-coordinate (left to right)
        eyes = sorted(eyes, key=lambda e: e[0])
        eye_centers = []
        
        for (ex, ey, ew, eh) in eyes[:2]:  # Take only the first two eyes
            eye_centers.append((x + ex + ew // 2, y + ey + eh // 2))

        # Draw glasses
        pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)
        
        # Draw circles around eyes
        for center in eye_centers:
            draw.ellipse(
                (center[0] - ew // 2, center[1] - eh // 2, center[0] + ew // 2, center[1] + eh // 2),
                outline=glasses_color,
                width=line_width
            )
        
        # Draw a line between the glasses with slight vertical adjustment
        midpoint_y = (eye_centers[0][1] + eye_centers[1][1]) // 2  # Centered vertically
        draw.line(
            (eye_centers[0][0] + ew // 2, midpoint_y, eye_centers[1][0] - ew // 2, midpoint_y),
            fill=glasses_color,
            width=line_width
        )
        
         # Draw angled brackets (ear arms) with an upward slope
        bracket_length = int(ew * 0.7)  # Adjusted length for a more natural look
        angle_offset = int(eh * 0.1)    # Adjust this value to control the upward angle
        
        # Left bracket
        draw.line(
            (eye_centers[0][0] - ew // 2, eye_centers[0][1], eye_centers[0][0] - ew // 2 - bracket_length, eye_centers[0][1] - angle_offset),
            fill=glasses_color,
            width=line_width
        )
        # Right bracket
        draw.line(
            (eye_centers[1][0] + ew // 2, eye_centers[1][1], eye_centers[1][0] + ew // 2 + bracket_length, eye_centers[1][1] - angle_offset),
            fill=glasses_color,
            width=line_width
        )

        pil_image = pil_image.resize(output_size, Image.LANCZOS)
        
        # Save the result
        pil_image.save(output_path)
        print(f"Saved output with adjusted ear-arm brackets to {output_path}")

draw_glasses_with_brackets(image_path, output_path_with_brackets, glasses_color, line_width)

Saved output with adjusted ear-arm brackets to ./labwork5dataset/longbottom_with_glasses_and_brackets.jpg
